# Double ML - modeling

`df_bp`

## 0. setup

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.preprocessing import LabelEncoder
import sklearn
import os
from matplotlib.pyplot import hist
import scipy.stats as stats
import math

In [2]:
# set random seed for numpy
RANDOM_SEED=42
np.random.seed(RANDOM_SEED)

In [3]:
def find_p(estimate, std):
    z_value = estimate / std
    p_value = stats.norm.sf(abs(z_value))*2
    return round(estimate, 4), round(std, 4), round(p_value, 4)

In [4]:
def label_encode_column(df, column):
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    return df

## 1. functions

### 1.1 Specify Nuisance Function Models

The next step is to specify models for 

*   $\mu(z,x)=\mathbb{E}(Y|z,x)$
*   $m(z,x) = P(A=1|z,x)$
*   $p(x) = P(Z=1|x)$

In [5]:
# make a function that returns a sklearn model for later use in k-folding
def make_mu_model():
  #return KNeighborsClassifier(n_neighbors=300)
  return RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=300, max_depth=None)
  #return RandomForestClassifier(n_estimators=100, max_depth=5)

# specify a model for m(z,x)
def make_m_model():
  #return LogisticRegression(max_iter=1000, warm_start=True, random_state=RANDOM_SEED)
  return RandomForestClassifier(n_estimators=200, max_depth=None)

def make_p_model():
  return RandomForestClassifier(n_estimators=200, max_depth=None) ###
  #return RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=300, max_depth=None)

### 1.2 Functions that use cross fitting to get predicted $\hat{\mu}$, $\hat{m}$, $\hat{p}$ for each unit

In [6]:
# helper functions to implement the cross fitting

def p_k_fold_fit_and_predict(make_model, X:pd.DataFrame, Z:np.array, n_splits:int):
    """
    Implements K fold cross-fitting for the model predicting the instrument Z. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns an array containing the predictions  

    Args:
    model: function that returns sklearn model (which implements fit and predict_prob)
    X: dataframe of variables to adjust for
    Z: array of instruments
    n_splits: number of splits to use
    """
    predictions = np.full_like(Z, np.nan, dtype=float)
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    
    for train_index, test_index in kf.split(X, Z):
      X_train = X.loc[train_index]
      Z_train = Z.loc[train_index]
      g = make_model()
      g.fit(X_train, Z_train)

      # get predictions for split
      predictions[test_index] = g.predict_proba(X.loc[test_index])[:, 1] ###
      # predictions[test_index] = g.predict(X.loc[test_index])

    assert np.isnan(predictions).sum() == 0
    return predictions


def m_k_fold_fit_and_predict(make_model, X:pd.DataFrame, Z:np.array, A:np.array, n_splits:int):
    """
    Implements K fold cross-fitting for the model predicting the outcome Y. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns two arrays containing the predictions for all units untreated, all units treated  

    Args:
    model: function that returns sklearn model (that implements fit and either predict_prob or predict)
    X: dataframe of variables to adjust for
    Z: array of instruments
    A: array of treatments
    n_splits: number of splits to use
    """
    predictions0 = np.full_like(A, np.nan, dtype=float)
    predictions1 = np.full_like(A, np.nan, dtype=float)
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)

    # include the treatment as input feature
    X_zx = X.copy()
    X_zx["Z"] = Z

    # for predicting A under Z=1 / Z=0 status for each data point 
    X0 = X_zx.copy()
    X0["Z"] = 0
    X1 = X_zx.copy()
    X1["Z"] = 1
    
    for train_index, test_index in kf.split(X_zx, A):
      X_train = X_zx.loc[train_index]
      A_train = A.loc[train_index]
      m = make_model()
      m.fit(X_train, A_train)
      predictions0[test_index] = m.predict_proba(X0.loc[test_index])[:,1]
      predictions1[test_index] = m.predict_proba(X1.loc[test_index])[:,1]

    assert np.isnan(predictions0).sum() == 0
    assert np.isnan(predictions1).sum() == 0
    return predictions0, predictions1

def mu_k_fold_fit_and_predict(make_model, X:pd.DataFrame, Z:np.array, y:np.array, n_splits:int, output_type:str):
    """
    Implements K fold cross-fitting for the model predicting the outcome Y. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns two arrays containing the predictions for all units untreated, all units treated  

    Args:
    model: function that returns sklearn model (that implements fit and either predict_prob or predict)
    X: dataframe of variables to adjust for
    Z: array of instruments
    y: array of outcomes
    n_splits: number of splits to use
    output_type: type of outcome, "binary" or "continuous"

    """
    predictions0 = np.full_like(y, np.nan, dtype=float)
    predictions1 = np.full_like(y, np.nan, dtype=float)
    if output_type == 'binary':
      kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    elif output_type == 'continuous':
      kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)

    # include the treatment as input feature
    X_zx = X.copy()
    X_zx["Z"] = Z

    # for predicting effect under treatment / control status for each data point 
    X0 = X_zx.copy()
    X0["Z"] = 0
    X1 = X_zx.copy()
    X1["Z"] = 1

    
    for train_index, test_index in kf.split(X_zx, y):
      X_train = X_zx.loc[train_index]
      y_train = y.loc[train_index]
      mu = make_model()
      mu.fit(X_train, y_train)

      if output_type =='binary':
        predictions0[test_index] = mu.predict_proba(X0.loc[test_index])[:, 1]
        predictions1[test_index] = mu.predict_proba(X1.loc[test_index])[:, 1]
      elif output_type == 'continuous':
        predictions0[test_index] = mu.predict(X0.loc[test_index])
        predictions1[test_index] = mu.predict(X1.loc[test_index])

    assert np.isnan(predictions0).sum() == 0
    assert np.isnan(predictions1).sum() == 0
    return predictions0, predictions1

### 1.3 LATE

In [7]:
def late_estimator(mu1, mu0, m1, m0, p, Z, A, Y, prob = None):
  '''
  Estimator for LATE
  '''
  n = len(Y)
  phi_zy = mu1 - mu0 + Z*(Y-mu1)/p - (1-Z)*(Y-mu0)/(1-p)
  phi_za = m1 - m0 + Z*(A-m1)/p - (1-Z)*(A-m0)/(1-p)

  tau_za = phi_za.mean()
  tau_hat = phi_zy.mean()/tau_za
  phi = phi_zy - phi_za * tau_hat
  
  std_hat = math.sqrt((phi**2).mean()/tau_za**2/n)

  return tau_hat, std_hat

### 1.4 Run a trial

In [8]:
def run(df, outcome_l, treatment_l, instrument_l, block_l, fe, stationary_c):

    df_1 = df[outcome_l+treatment_l+instrument_l+block_l]
    df_1 = df_1.dropna()

    outcome = df_1[outcome_l].reset_index(drop=True).squeeze()
    treatment = df_1[treatment_l].reset_index(drop=True).squeeze()
    instrument = df_1[instrument_l].reset_index(drop=True).squeeze()
    block = df_1[block_l].reset_index(drop=True)

    p = p_k_fold_fit_and_predict(make_p_model, X=block, Z=instrument, n_splits=10)
    m0,m1= m_k_fold_fit_and_predict(make_m_model, X=block, Z=instrument, A=treatment, n_splits=10)
    mu0,mu1= mu_k_fold_fit_and_predict(make_mu_model, X=block, Z=instrument, y=outcome, n_splits=10, output_type="continuous")
    tau_hat, std_hat = late_estimator(mu1, mu0, m1, m0, p, Z=instrument, A=treatment, Y=outcome, prob = None)
    p, tau_hat, sd_hat = find_p(tau_hat, std_hat)
    
    return outcome_l[0], treatment_l[0], instrument_l[0], p, tau_hat, sd_hat, fe, stationary_c
    

## 2. Analysis

### 2.1 `df_bp`

In [9]:
# read in the dataframe
df = pd.read_csv('df_bp.csv')

In [10]:
df1 = pd.read_csv('../../data/GVC_data/transportIV_file.csv')
df1 = df1.loc[:, ['country', 't', 'trans_outp_p']]

df = pd.merge(df, df1, on=['country', 't'])

df = df.drop(columns='iv_transport')
df = df.rename(columns={'trans_outp_p': 'iv_transport'})

# Define categorization function
def categorize_value(value, q1_3, q2_3):
    if value > q2_3:
        return 1
    elif value < q1_3:
        return 0
    else:
        return np.nan

# Columns to apply the transformation
columns = ['iv_transport']

# Iterate through the columns and apply the categorization function
for col in columns:
    q1_3 = df[col].quantile(1/3)
    q2_3 = df[col].quantile(2/3)
    
    df[col] = df[col].apply(lambda x: categorize_value(x, q1_3, q2_3))

In [11]:
# in order to run random forest with categorical variable
df = label_encode_column(df, 'country')

In [12]:
res = pd.DataFrame(columns=['outcome', 'treatment', 'instrument', 'tau_hat', 'std_hat', 'p_val', 'fixed_effects', 'stationary_controls'])

In [13]:
df.columns

Index(['country', 't', 'onset2COWCS', 'decade', 'democracy', 'logmountain',
       'ethnic_fractionalization', 'religion_fractionalization',
       'language_fractionalization', 'leg_british', 'opec', 'logpop_M_diff',
       'logpopdens_diff', 'logoutreg_diff', 'ecgrowth_demeaned', 'treat_agri',
       'treat_mine', 'treat_fuel', 'treat_metal', 'iv_agri', 'iv_mine',
       'iv_fuel', 'iv_metal', 'iv_transport'],
      dtype='object')

In [14]:
def run_all(df, outcome_l, treatment_l, instrument_l, block_fe_l, block_sta_l, block_other_l):
    '''
    For a given treatment i.e. sector.

    instrument_l: a list of instruments.
    '''
    res = pd.DataFrame(columns=['outcome', 'treatment', 'instrument', 'tau_hat', 'std_hat', 'p_val', 'fixed_effects', 'stationary_controls'])

    for ins_l in instrument_l:
        block_l = block_other_l

        for fe in [True, False]:
            if fe:
                block_l += block_fe_l

            for sta in [True, False]:
                if sta:
                    block_l += block_sta_l
                res_row = run(df, outcome_l, treatment_l, ins_l, block_l, fe, sta)
                res.loc[len(res)] = list(res_row)
    return res


Treatment: Fuel Sector

In [15]:
fuel_res = run_all(df, 
        outcome_l = ['onset2COWCS'], 
        treatment_l = ['treat_fuel'], 
        instrument_l = [['iv_transport'], ['iv_fuel']], 
        block_fe_l = ['country', 't'], 
        block_sta_l = ['logmountain', 'ethnic_fractionalization', 'religion_fractionalization',
                       'language_fractionalization', 'leg_british', 'opec'], 
        block_other_l = ['democracy', 'logpop_M_diff', 'logpopdens_diff', 
                         'logoutreg_diff', 'ecgrowth_demeaned'])

In [16]:
fuel_res

,outcome,treatment,instrument,tau_hat,std_hat,p_val,fixed_effects,stationary_controls
0,onset2COWCS,treat_fuel,iv_transport,0.3189,0.8751,0.7155,True,True
1,onset2COWCS,treat_fuel,iv_transport,0.3039,0.7266,0.6758,True,False
2,onset2COWCS,treat_fuel,iv_transport,0.2728,0.3844,0.4779,False,True
3,onset2COWCS,treat_fuel,iv_transport,0.1710,0.2506,0.4951,False,False
4,onset2COWCS,treat_fuel,iv_fuel,-0.0611,0.2891,0.8326,True,True
5,onset2COWCS,treat_fuel,iv_fuel,-0.0523,0.2726,0.8479,True,False
6,onset2COWCS,treat_fuel,iv_fuel,-0.0530,0.2642,0.8409,False,True
7,onset2COWCS,treat_fuel,iv_fuel,-0.0546,0.2932,0.8522,False,False


Treatment: Agriculture

In [17]:
agri_res = run_all(df, 
        outcome_l = ['onset2COWCS'], 
        treatment_l = ['treat_agri'], 
        instrument_l = [['iv_transport'], ['iv_agri']], 
        block_fe_l = ['country', 't'], 
        block_sta_l = ['logmountain', 'ethnic_fractionalization', 'religion_fractionalization',
                       'language_fractionalization', 'leg_british', 'opec'], 
        block_other_l = ['democracy', 'logpop_M_diff', 'logpopdens_diff', 
                         'logoutreg_diff', 'ecgrowth_demeaned'])

In [18]:
agri_res


,outcome,treatment,instrument,tau_hat,std_hat,p_val,fixed_effects,stationary_controls
0,onset2COWCS,treat_agri,iv_transport,0.0349,0.2151,0.8712,True,True
1,onset2COWCS,treat_agri,iv_transport,0.0363,0.2461,0.8826,True,False
2,onset2COWCS,treat_agri,iv_transport,0.3111,0.5066,0.5392,False,True
3,onset2COWCS,treat_agri,iv_transport,0.1189,0.2985,0.6904,False,False
4,onset2COWCS,treat_agri,iv_agri,-0.0485,0.0275,0.0775,True,True
5,onset2COWCS,treat_agri,iv_agri,-0.0519,0.0287,0.0703,True,False
6,onset2COWCS,treat_agri,iv_agri,-0.0509,0.0299,0.0883,False,True
7,onset2COWCS,treat_agri,iv_agri,-0.0547,0.0314,0.0816,False,False


Treatment: Metal

In [19]:
metal_res = run_all(df, 
        outcome_l = ['onset2COWCS'], 
        treatment_l = ['treat_metal'], 
        instrument_l = [['iv_transport'], ['iv_metal']], 
        block_fe_l = ['country', 't'], 
        block_sta_l = ['logmountain', 'ethnic_fractionalization', 'religion_fractionalization',
                       'language_fractionalization', 'leg_british', 'opec'], 
        block_other_l = ['democracy', 'logpop_M_diff', 'logpopdens_diff', 
                         'logoutreg_diff', 'ecgrowth_demeaned'])

In [20]:
metal_res

,outcome,treatment,instrument,tau_hat,std_hat,p_val,fixed_effects,stationary_controls
0,onset2COWCS,treat_metal,iv_transport,-0.0338,0.1481,0.8194,True,True
1,onset2COWCS,treat_metal,iv_transport,0.0196,0.2644,0.9409,True,False
2,onset2COWCS,treat_metal,iv_transport,-0.0794,0.2031,0.6959,False,True
3,onset2COWCS,treat_metal,iv_transport,-0.0153,0.2945,0.9584,False,False
4,onset2COWCS,treat_metal,iv_metal,-0.0046,0.0258,0.8598,True,True
5,onset2COWCS,treat_metal,iv_metal,-0.0030,0.0260,0.9069,True,False
6,onset2COWCS,treat_metal,iv_metal,-0.0062,0.0277,0.8229,False,True
7,onset2COWCS,treat_metal,iv_metal,-0.0051,0.0282,0.8559,False,False


Treatment: Mining

In [21]:
mine_res = run_all(df, 
        outcome_l = ['onset2COWCS'], 
        treatment_l = ['treat_mine'], 
        instrument_l = [['iv_transport'], ['iv_mine']], 
        block_fe_l = ['country', 't'], 
        block_sta_l = ['logmountain', 'ethnic_fractionalization', 'religion_fractionalization',
                       'language_fractionalization', 'leg_british', 'opec'], 
        block_other_l = ['democracy', 'logpop_M_diff', 'logpopdens_diff', 
                         'logoutreg_diff', 'ecgrowth_demeaned'])

In [22]:
mine_res

,outcome,treatment,instrument,tau_hat,std_hat,p_val,fixed_effects,stationary_controls
0,onset2COWCS,treat_mine,iv_transport,-0.4274,3.2926,0.8967,True,True
1,onset2COWCS,treat_mine,iv_transport,-4.3942,354.4949,0.9901,True,False
2,onset2COWCS,treat_mine,iv_transport,0.0040,0.9766,0.9967,False,True
3,onset2COWCS,treat_mine,iv_transport,0.0754,0.3636,0.8358,False,False
4,onset2COWCS,treat_mine,iv_mine,-0.0221,0.0361,0.5408,True,True
5,onset2COWCS,treat_mine,iv_mine,-0.0244,0.0350,0.4851,True,False
6,onset2COWCS,treat_mine,iv_mine,-0.0255,0.0360,0.4783,False,True
7,onset2COWCS,treat_mine,iv_mine,-0.0238,0.0370,0.5194,False,False


Final Result:

In [23]:
dfs = [fuel_res, agri_res, metal_res, mine_res]
stacked_df = pd.concat(dfs)
final_res = stacked_df.reset_index(drop=True)

In [24]:
final_res.insert(0, 'gvc_type', 'backward')
final_res

,gvc_type,outcome,treatment,instrument,tau_hat,std_hat,p_val,fixed_effects,stationary_controls
0,backward,onset2COWCS,treat_fuel,iv_transport,0.3189,0.8751,0.7155,True,True
1,backward,onset2COWCS,treat_fuel,iv_transport,0.3039,0.7266,0.6758,True,False
2,backward,onset2COWCS,treat_fuel,iv_transport,0.2728,0.3844,0.4779,False,True
3,backward,onset2COWCS,treat_fuel,iv_transport,0.1710,0.2506,0.4951,False,False
4,backward,onset2COWCS,treat_fuel,iv_fuel,-0.0611,0.2891,0.8326,True,True
5,backward,onset2COWCS,treat_fuel,iv_fuel,-0.0523,0.2726,0.8479,True,False
6,backward,onset2COWCS,treat_fuel,iv_fuel,-0.0530,0.2642,0.8409,False,True
7,backward,onset2COWCS,treat_fuel,iv_fuel,-0.0546,0.2932,0.8522,False,False
8,backward,onset2COWCS,treat_agri,iv_transport,0.0349,0.2151,0.8712,True,True
9,backward,onset2COWCS,treat_agri,iv_transport,0.0363,0.2461,0.8826,True,False


In [25]:
final_res.to_csv('backward_res.csv', index=False)

For estimating the local average treatment effect under the monotone instrument assumption, there is a double-machine learning approach that works with generic supervised learning approaches. Here, we want an estimator $\hat{\tau}^{\mathrm{LATE}}$ for the parameter
$$
\tau^{\mathrm{LATE}}=\frac{\mathbb{E}[\mathbb{E}[Y \mid X, Z=1]-\mathbb{E}[Y \mid X, Z=0]]}{\mathbb{E}[\mathrm{P}(A=1 \mid X, Z=1)-\mathrm{P}(A=1 \mid X, Z=0)]}
$$
To define the estimator, it's convenient to introduce some additional notation. First, we define the nuisance functions:
$$
\begin{aligned}
\mu(z, x) & =\mathbb{E}[Y \mid z, x] \\
m(z, x) & =\mathrm{P}(A=1 \mid x, z) \\
p(x) & =\mathrm{P}(Z=1 \mid x) .
\end{aligned}
$$
We also define the score $\phi$ by:
$$
\begin{aligned}
& \phi_{Z \rightarrow Y}(\mathbf{X} ; \mu, p) \triangleq \mu(1, X)-\mu(0, X)+\frac{Z(Y-\mu(1, X))}{p(X)}-\frac{(1-Z)(Y-\mu(0, X))}{1-p(X)} \\
& \phi_{Z \rightarrow A}(\mathbf{X} ; m, p) \triangleq m(1, X)-m(0, X)+\frac{Z(A-m(1, X))}{p(X)}-\frac{(1-Z)(A-m(0, X))}{1-p(X)} \\
& \phi(\mathbf{X} ; \mu, m, p, \tau) \triangleq \phi_{Z \rightarrow Y}(\mathbf{X} ; \mu, p)-\phi_{Z \rightarrow A}(\mathbf{X} ; m, p) \times \tau
\end{aligned}
$$
Then, the estimator is defined by a two stage procedure:
1. Fit models $\hat{\mu}, \hat{m}, \hat{p}$ for each of $\mu, m, p$ (using supervised machine learning).
2. Define $\hat{\tau}^{\mathrm{LATE}}$ as the solution to $\frac{1}{n} \sum_i \phi\left(\mathbf{X}_i ; \hat{\mu}, \hat{m}, \hat{p}, \hat{\tau}^{\mathrm{LATE}}\right)=0$. That is,
$$
\hat{\tau}^{\mathrm{LATE}}=\frac{\frac{1}{n} \sum_i \phi_{Z \rightarrow Y}\left(\mathbf{X}_i ; \hat{\mu}, \hat{p}\right)}{\frac{1}{n} \sum_i \phi_{Z \rightarrow A}\left(\mathbf{X}_i ; \hat{m}, \hat{p}\right)}
$$
It may help intuitions to notice that the double machine learning estimator of the LATE is effectively the double machine learning estimator of of the average treatment effect of $Z$ on $Y$ divided by the double machine learning estimator of the average treatment effect of $Z$ on $A$.
The nuisance functions can be estimated by:
1. fit a model $\hat{\mu}$ that predicts $Y$ from $Z, X$ by minimizing mean square error
2. fit a model $\hat{m}$ that predicts $A$ from $Z, X$ by minimizing mean cross-entropy
3. fit a model $\hat{p}$ that predicts $Z$ from $X$ by minimizing mean cross-entropy.